# Predicting Click-Through's for Recommended Content

## Reduce Kaggle Data Set to US only
The data set used for the kaggle competition was large in scope. For this project I am scaling back the data to work on a local machine. After intial runs of this data, further work can be done by moving to larger systems to run the models.


## Sampling for Train and Test data & merging tables
Given the size and scope of the data, and having done the initial EDA, the preliminary model is to be developed using a random subset of the data.

#### The following factors are being applied to pair down the data
* Only using US data
* This is a predominant part of the data, and EDA shows not much variation in behaviour in regards to display count and other factors.
* Of this data, only those with full geo_location data will be used. This is to ensure proper correction for time zones (base times are in UTC counts).
* From this a random slice of events will be chosen.

The initial training data will be 5,000 events and the test set will be 500 events. Since each event has on average around five ad choices, this will tranlate in 25,000 rows of data for developing the training set.

#### Further refinements to the process will include what selection of document data to use
* Much of the data has low confidence levels
* Will need to develop a threshold for when to use and when to ignore confidence level
* Many documents also have multiple categories, so will need to determine how to pick the main one to use, or as the model develops, how to incorporate multiple categories.

## Slicing and prepping the data

Note: CAPITAL LETTER items reference tables provided by Outbrains.

Process:
* Read in Outbrain CLICK data in parts, too big to read in as one dataframe
* Compare to EVENTS data supplied by outbrain (Events is larger set the Click data)
* EVENTS - make 2nd version that:
    * Slice EVENTS down so that it only has items that are in CLICK
    * Split up geo_location column into 'states' and 'DMA'
    * Filter out empties and non US data, as well as data missing state and DMA info


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import random
import matplotlib
import matplotlib.cm as cm
import seaborn as sns
import datetime
import time

matplotlib.style.use('ggplot') 
%matplotlib inline

## Read in Event Table Data

The event file contains the item clicked (not the other ads displayed) and some basic user information.


In [2]:
# !ls ./data/ 

In [3]:
events = pd.read_csv('./data/events.csv')

/Users/alex/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print('Total number of events in full data: {0:,}'.\
      format(events['display_id'].nunique()))

Total number of events in full data: 23,120,126


## Read the Page Recommended Content Data into 2 data frames
File is too large to read into one dataframe. This file contains all the ads displayed at each click event. Each selection of ads is denoted by 'display_id'.

In [5]:
df1 = pd.read_csv('./data/clicks_train.csv', nrows = 40000000)
df2 = pd.read_csv('./data/clicks_train.csv', skiprows = 40000000)

In [6]:
titles = df1.columns
df2.columns = titles

In [7]:
df2.head(2)

,display_id,ad_id,clicked
0,7777790,152975,0
1,7777790,159143,0


In [8]:
df1.head(2)

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0


## Do some data cleaning

In [9]:
print(events['platform'].unique())
events['platform'].value_counts()

[3 2 1 '2' '1' '3' '\\N']


2     10684579
1      8747280
3      3032907
2       291699
1       279988
3        83668
\N           5
Name: platform, dtype: int64

In [10]:
# function to convert all data to strings (will be used for classification)

def platform_filter(x):
    if x == '\\N':
        return np.nan
    elif type(x) is int:
        return str(x)  # make into string since this is a category
    else:
        return x

# updated platform column
events['platform'] = [platform_filter(x) for x in events['platform']]
events['platform'].unique()

array(['3', '2', '1', nan], dtype=object)

## Break out Geo_Location data and use only US data

In [11]:
events['country'] = events['geo_location'].str[0:2]
# take only US data from events
# and break out other geographic data
events = events[events['country'] == 'US']
events['state'] = events['geo_location'].str[3:5]
events['DMA'] = events['geo_location'].str[6:]

In [12]:
events.head(3)

,display_id,uuid,document_id,timestamp,platform,geo_location,country,state,DMA
0,1,cb8c55702adb93,379743,61,3,US>SC>519,US,SC,519
1,2,79a85fa78311b9,1794259,81,2,US>CA>807,US,CA,807
2,3,822932ce3d8757,1179111,182,2,US>MI>505,US,MI,505


In [13]:
# Clean DMA and state columns
def filter_DMA(x):
    try:
        return str(int(x))
    except:
        return np.nan

def filter_state(x):
    if x == '':
        return np.nan
    else:
        return x
    
events['state'] = [filter_state(x) for x in events['state']]
events['DMA'] = [filter_DMA(x) for x in events['DMA']]

In [14]:
print('# rows mising state data: {0:,}'.format(len(events[events['state'].isnull()])))
print('# rows mising platform data: {0:,}'.format( len(events[events['platform'].isnull()])))
print('# rows mising DMA data: {0:,}'.format( len(events[events['DMA'].isnull()])))

# rows mising state data: 758,487
# rows mising platform data: 5
# rows mising DMA data: 1,264,852


In [15]:
print('# of total US events: {0:,}'.format(len(events)))

# of total US events: 18,595,452


In [16]:
US_orig_length = len(events)

### Take out events missing geo data & military location designators
AA and AP are military postal designations for overseas.

In [17]:
events = events[(events['state'].notnull()) & (events['DMA'].notnull()) \
                & (events['state'] != "AA") & (events['state'] != 'AP') \
                & (events['platform'].notnull())]


In [18]:
print('# rows mising state data: {0:,}'.format(len(events[events['state'].isnull()])))
print('# rows mising platform data: {0:,}'.format( len(events[events['platform'].isnull()])))
print('# rows mising DMA data: {0:,}'.format( len(events[events['DMA'].isnull()])))

# rows mising state data: 0
# rows mising platform data: 0
# rows mising DMA data: 0


In [19]:
reduction1 = US_orig_length - len(events)
print('# of rows reduced: {0:,}'.format(reduction1))

# of rows reduced: 1,264,857


All the missing state data came from rows missing the DMA data (the 5 stragglers may be from misssing platform data or DMA or both)

### Sync Data Between Main Click Data and EVENT table
EVENT has more click's than in the main data set, so we want to reduce EVENT table to only have correpsonding click events.

In [20]:
# identify events that are in main click data set 
event_id = list(df1['display_id'].unique()) + list(df2['display_id'].unique())

In [21]:
# filter main events dataframe to only include event_id's in main click data set
events = events[events['display_id'].isin(event_id)]

In [22]:
print('Further adjustment to number of events: {0:,}'.\
      format(US_orig_length - reduction1 - len(events)))

Further adjustment to number of events: 4,652,848


Check events dataframe for stray nan or other odd values (ok since nothing odd at begining or end)

In [23]:
events['uuid'].unique()

array(['cb8c55702adb93', '79a85fa78311b9', '822932ce3d8757', ...,
       '4032cf074d74a3', '49396799cb3a40', '21f03d8a66e702'], dtype=object)

In [24]:
events['document_id'].unique()

array([ 379743, 1794259, 1179111, ..., 2681997, 2788096,  682611])

In [25]:
events['timestamp'].unique()

array([        61,         81,        182, ..., 1123199470, 1123199601,
       1123199936])

### Take list from events['display_id'] to generate random sample list
This will be used later to draw random samples from the assembled dataset.

In [26]:
event_US_id = list(events['display_id'].unique())

## Date - put time stamp in terms of dates and local times

In [27]:
# to adjsut timestamp later
from datetime import datetime
datetime.fromtimestamp(1465876800)
# the timestamp is adjusted to time zero for data colleciton, whcih is 6/14/2016
# adj in milliseconds: 1465876799998ms

datetime.datetime(2016, 6, 14, 0, 0)

In [28]:
tz= pd.read_csv('./data/tz.csv')
tz.head()

,state,tz,utc_summer,tz_adj_sec
0,AK,AKST,-8,-28800
1,AL,CST,-5,-18000
2,AR,CST,-5,-18000
3,IL,CST,-5,-18000
4,IA,CST,-5,-18000


In [29]:
events = events.merge(tz,on='state',how='left', suffixes=('_l','_r'))

In [30]:
events['ts_UTC'] = [t + 1465876799998 for t in events['timestamp']]
events['date_UTC'] = pd.to_datetime(events['ts_UTC'],unit='ms')

In [31]:
events['ts_local'] = events['ts_UTC'] + 1000*events['tz_adj_sec']  # assigned to a column
events['date_local'] = pd.to_datetime(events['ts_local'],unit='ms')
events.tail()

,display_id,uuid,document_id,timestamp,platform,geo_location,country,state,DMA,tz,utc_summer,tz_adj_sec,ts_UTC,date_UTC,ts_local,date_local
12677742,16874588,1bf30bbd832319,2822648,1123199298,2,US>VA>511,US,VA,511,EST,-4,-14400,1466999999296,2016-06-27 03:59:59.296,1466985599296,2016-06-26 23:59:59.296
12677743,16874589,d2c47d8183e37b,876520,1123199313,3,US>TN>557,US,TN,557,CST,-5,-18000,1466999999311,2016-06-27 03:59:59.311,1466981999311,2016-06-26 22:59:59.311
12677744,16874590,4032cf074d74a3,2819923,1123199470,3,US>NM>790,US,NM,790,MST,-6,-21600,1466999999468,2016-06-27 03:59:59.468,1466978399468,2016-06-26 21:59:59.468
12677745,16874591,49396799cb3a40,2816969,1123199601,1,US>IN>582,US,IN,582,EST,-4,-14400,1466999999599,2016-06-27 03:59:59.599,1466985599599,2016-06-26 23:59:59.599
12677746,16874593,21f03d8a66e702,2777166,1123199936,2,US>NJ>501,US,NJ,501,EST,-4,-14400,1466999999934,2016-06-27 03:59:59.934,1466985599934,2016-06-26 23:59:59.934


In [32]:
events.drop(['utc_summer','timestamp','tz','utc_summer','tz_adj_sec','ts_UTC'],\
            axis=1, inplace=True)

In [33]:
events.drop(['date_UTC','ts_local'], axis=1, inplace=True)

In [34]:
events.head()

,display_id,uuid,document_id,platform,geo_location,country,state,DMA,date_local
0,1,cb8c55702adb93,379743,3,US>SC>519,US,SC,519,2016-06-14 00:00:00.059
1,2,79a85fa78311b9,1794259,2,US>CA>807,US,CA,807,2016-06-13 21:00:00.079
2,3,822932ce3d8757,1179111,2,US>MI>505,US,MI,505,2016-06-14 00:00:00.180
3,4,85281d0a49f7ac,1777797,2,US>WV>564,US,WV,564,2016-06-14 00:00:00.232
4,6,7765b4faae4ad4,1773517,3,US>OH>510,US,OH,510,2016-06-14 00:00:00.393


In [35]:
def time_bucket(x):
    hour = int(str(x)[11:13])
    if hour >6 and hour <12:
        s = 'morning'
    elif hour > 11 and hour <18:
        s = 'afternoon'
    elif hour > 17 and hour < 23:
        s = 'evening'
    else:
        s = 'overnight'
    return s

def date_bucket(x):
    return int(str(x)[8:10]) # this is the date space in datetime

def hour(x):
    return int(str(x)[11:13])

In [36]:
str(events['date_local'][4])[8:10]

'14'

In [37]:
events.head(1)

,display_id,uuid,document_id,platform,geo_location,country,state,DMA,date_local
0,1,cb8c55702adb93,379743,3,US>SC>519,US,SC,519,2016-06-14 00:00:00.059


In [38]:
events['period'] = [time_bucket(rows['date_local']) for index,\
                        rows in events.iterrows()]
events['day_of_mnth'] = [date_bucket(rows['date_local']) for index,\
                        rows in events.iterrows()]

In [39]:
events.head()

,display_id,uuid,document_id,platform,geo_location,country,state,DMA,date_local,period,day_of_mnth
0,1,cb8c55702adb93,379743,3,US>SC>519,US,SC,519,2016-06-14 00:00:00.059,overnight,14
1,2,79a85fa78311b9,1794259,2,US>CA>807,US,CA,807,2016-06-13 21:00:00.079,evening,13
2,3,822932ce3d8757,1179111,2,US>MI>505,US,MI,505,2016-06-14 00:00:00.180,overnight,14
3,4,85281d0a49f7ac,1777797,2,US>WV>564,US,WV,564,2016-06-14 00:00:00.232,overnight,14
4,6,7765b4faae4ad4,1773517,3,US>OH>510,US,OH,510,2016-06-14 00:00:00.393,overnight,14


In [40]:
start = datetime.now()

events['hour'] = [hour(rows['date_local']) for index,\
                        rows in events.iterrows()]

finish = datetime.now()
print('Time to run: {0}'.format(finish - start))

Time to run: 0:22:36.867717


In [41]:
import os
os.system('say "Hour run finished!"')

0

In [42]:
start = datetime.now()

events.to_csv('./data/events_b1a.csv',index=False)

finish = datetime.now()
print('Time to run: {0}'.format(finish - start))

Time to run: 0:01:25.210845


In [43]:
# events = pd.read_csv('./data/events_b1a.csv')

In [44]:
events.head()

,display_id,uuid,document_id,platform,geo_location,country,state,DMA,date_local,period,day_of_mnth,hour
0,1,cb8c55702adb93,379743,3,US>SC>519,US,SC,519,2016-06-14 00:00:00.059,overnight,14,0
1,2,79a85fa78311b9,1794259,2,US>CA>807,US,CA,807,2016-06-13 21:00:00.079,evening,13,21
2,3,822932ce3d8757,1179111,2,US>MI>505,US,MI,505,2016-06-14 00:00:00.180,overnight,14,0
3,4,85281d0a49f7ac,1777797,2,US>WV>564,US,WV,564,2016-06-14 00:00:00.232,overnight,14,0
4,6,7765b4faae4ad4,1773517,3,US>OH>510,US,OH,510,2016-06-14 00:00:00.393,overnight,14,0


In [45]:
import os
os.system('say "Hudson is cool!"')

0

## Now slice and dice main data and events data to what is needed to run models

This will be 10,000 for training data and 1,000 for test data

* Random list of 11,000
* Filter EVENTS
    * Done a few steps above
* Filter df1 and df2 for 11k id's in sample list
    * Merge now that down to manageable size
    * Split into train and test
    * Save to .csv
    


In [46]:
disp_list = list(events['display_id'].unique())
# cut down list to 15,000 for sampling

# Random list from event_id list of df1 & df2
random.seed(a=47)
rand_list = random.sample(disp_list,15000)
len(rand_list)

15000

In [47]:
data1 = df1[df1['display_id'].isin(rand_list)]
data2 = df2[df2['display_id'].isin(rand_list)]


In [48]:
data = pd.concat([data1,data2], axis=0)


In [49]:
data.head(3)

,display_id,ad_id,clicked
7719,1525,8846,0
7720,1525,45180,0
7721,1525,46293,0


In [50]:
print('Number of events in train data: {0:,}'.format(data['display_id'].nunique()))
print('Number of rows (total ads shown) in train data: {0:,}'.\
      format(len(data['display_id'])))

Number of events in train data: 15,000
Number of rows (total ads shown) in train data: 77,226


In [51]:
# clean up dataframes no longer being used:
del df1, df2, data1, data2

## Merge click and event data into main dataframe

In [52]:
data = data.merge(events,on='display_id',how='left', suffixes=('_l','_r'))

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77226 entries, 0 to 77225
Data columns (total 14 columns):
display_id      77226 non-null int64
ad_id           77226 non-null int64
clicked         77226 non-null int64
uuid            77226 non-null object
document_id     77226 non-null int64
platform        77226 non-null object
geo_location    77226 non-null object
country         77226 non-null object
state           77226 non-null object
DMA             77226 non-null object
date_local      77226 non-null datetime64[ns]
period          77226 non-null object
day_of_mnth     77226 non-null int64
hour            77226 non-null int64
dtypes: datetime64[ns](1), int64(6), object(7)
memory usage: 8.8+ MB


In [54]:
data.head(2)

,display_id,ad_id,clicked,uuid,document_id,platform,geo_location,country,state,DMA,date_local,period,day_of_mnth,hour
0,1525,8846,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23
1,1525,45180,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23


In [55]:
pages = data['display_id'].unique()
len(pages)

15000

In [56]:
data['ads_per_page'] = [0 for _ in range(len(data))] # puts zeros in place

start = datetime.now()

for d_num in pages:  
    disp_list = data[data['display_id'] == d_num].index.tolist()
    size = len(disp_list)
    for n in disp_list:
        data.set_value(n, 'ads_per_page', size)

finish = datetime.now()
print('Time to run: {0}'.format(finish - start))
os.system('say "I am finished!"')

Time to run: 0:00:10.037552


0

In [57]:
data.head()

,display_id,ad_id,clicked,uuid,document_id,platform,geo_location,country,state,DMA,date_local,period,day_of_mnth,hour,ads_per_page
0,1525,8846,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23,9
1,1525,45180,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23,9
2,1525,46293,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23,9
3,1525,77878,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23,9
4,1525,78011,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23,9


## Bring in promoted content info
This table ties the ad_id from the click data to the ad's document_id, the advertiser and the campaign. The document_id is key to pulling topical/categorical data later.

In [58]:
pc = pd.read_csv('./data/promoted_content.csv')
pc.head()

,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7
3,4,471471,2,7
4,5,471472,2,7


In [59]:
data = data.merge(pc,on='ad_id',how='left', suffixes=('_l','_r'))


In [60]:
data.rename(columns={'document_id_l':'document_id','document_id_r':'ad_document_id' },\
                    inplace=True)

In [61]:
data[(data['document_id'] == 1047732) &\
      (data['ad_document_id'] == 700394)]


,display_id,ad_id,clicked,uuid,document_id,platform,geo_location,country,state,DMA,date_local,period,day_of_mnth,hour,ads_per_page,ad_document_id,campaign_id,advertiser_id
10,1796,14120,0,6ebcd6c9c6bc96,1047732,2,US>CA>807,US,CA,807,2016-06-13 21:02:05.717,evening,13,21,4,700394,2496,1222


## Bring in Document Data
There are three sets of document data that attempt to classify the material inside a document page:
* Topic
* Category
* Entity <br>

Some documents have more than one of the above for categorization, in those cases the item with the highest confidence level will be added to the table.

In [62]:
d_top = pd.read_csv('./data/documents_topics.csv')
# d_cat = pd.read_csv('./data/documents_categories.csv')
# d_ent = pd.read_csv('./data/documents_entities.csv')

An example of how data is organized in the tables.
* document_id of source page: 1773554
* document_id's of ad pages:
    * 1328059
    * 1086095
    * 1414815
    * 1130922

For the source page, topic 199 will be chosen, for the first ad page 160 will be chose, and since the remainder have only one choice that will be what is used.

In [63]:
# the original page document_id
d_top[d_top['document_id'] == 1773554]

,document_id,topic_id,confidence_level
8371044,1773554,199,0.090703
8371045,1773554,183,0.013426
8371046,1773554,133,0.008202


In [64]:
# the corresponding ad doc_id's
d_top[d_top['document_id'] == 1328059]

,document_id,topic_id,confidence_level
10330835,1328059,160,0.331205
10330836,1328059,66,0.008115


In [65]:
d_top[d_top['document_id'] == 1086095]

,document_id,topic_id,confidence_level
10588629,1086095,26,0.259674


In [66]:
d_top[d_top['document_id'] == 1414815]

,document_id,topic_id,confidence_level
9395631,1414815,183,0.309797


In [67]:
d_top[d_top['document_id'] == 1130922]

,document_id,topic_id,confidence_level
10551623,1130922,107,0.261741


In [68]:
# function to find the max confidence level 
def doc_max (x,df,df_col):
    try:
        d_index = df[df['document_id'] == x][df_col].idxmax(axis=0, skipna=True)
        return d_top.iloc[d_index][df_col]
    except:
        pass
    else:
        return np.nan


In [69]:
start = datetime.now()

data['doc_id_topic'] = [doc_max(rows['document_id'],d_top,'topic_id') for index, \
                                rows in data.iterrows()]


finish = datetime.now()
print('Time to run: {0}'.format(finish - start))
os.system('say "I am finished!"')

Time to run: 0:15:45.449636


0

In [70]:
data['doc_id_topic'].isnull().sum()

7388

About 10% of sample set does not have doc_id, to critical to be imputed so will continue with reduced dataset and work on remediation on next round of analysis.

In [71]:
start = datetime.now()

data['ad_id_topic'] = [doc_max(rows['ad_document_id'],d_top,'topic_id') for index, \
                                rows in data.iterrows()]


finish = datetime.now()
print('Time to run: {0}'.format(finish - start))
os.system('say "I am finished!"')

Time to run: 0:19:22.887628


0

<font color='red'> **Not all documents are showing an topic, about half are missing. Will still include **

In [72]:
data['display_id'].nunique()

15000

In [73]:
data.head(3)

,display_id,ad_id,clicked,uuid,document_id,platform,geo_location,country,state,DMA,date_local,period,day_of_mnth,hour,ads_per_page,ad_document_id,campaign_id,advertiser_id,doc_id_topic,ad_id_topic
0,1525,8846,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23,9,21135,1726,24,271.0,125.0
1,1525,45180,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23,9,953930,6296,1594,271.0,286.0
2,1525,46293,0,443715a4da754f,1697627,1,US>IL>602,US,IL,602,2016-06-13 23:01:46.019,overnight,13,23,9,959753,959,8,271.0,147.0


In [74]:
id_list = []

for i,r in data.iterrows():
    if np.isnan(r['doc_id_topic']):
        id_list.append(r['display_id'])
#pd.isnull(data.doc_id_topic).sum()


In [75]:
missing_id = list(set(id_list))
len(missing_id)
good_id = [x for x in pages if x not in missing_id]

In [76]:
len(good_id)

13574

# Take out display_id's missing doc_id_topic

This item is categorical and would be complicated to impute at this point.

In [77]:
data2 = data[data['display_id'].isin(good_id)]

In [78]:
len(data2)

69838

In [79]:
data2['display_id'].nunique()

13574

# Save tables to .csv

In [80]:
data2.to_csv('./data/data_bk1_complete.csv',index=False)


In [81]:
with open('./data/data_bk1_complete.pkl', 'w') as picklefile:
    pickle.dump(data2, picklefile)